In [1]:
import json
from functools import partial
from typing import List, Set
import os
import pickle


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

In [ ]:
# from distutils.dir_util import copy_tree
# copy_tree("/kaggle/input/hackathon-files-for-participants-ozon", "/kaggle/working/hackathon_files_for_participants_ozon")

### Load data

In [2]:
dataset = pd.read_parquet("hackathon_files_for_participants_ozon/train_pairs.parquet")
etl = pd.read_parquet("hackathon_files_for_participants_ozon/train_data.parquet")

In [7]:
dataset.head(2)

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289


In [8]:
dataset["target"].value_counts()

0.0    171527
1.0    135013
Name: target, dtype: int64

In [9]:
etl.head(3)

,variantid,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
0,51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...","{""Номинальный ток, А"":[""10""],""Цвет товара"":[""о..."
1,53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Кабели ...",[красный],"[[0.26863545, -0.3130674, 0.29023397, 0.073978...","[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...","{""Конструктивные особенности"":[""Магнитная конс..."
2,56763357,"Набор микропрепаратов Konus 25: ""Клетки и ткан...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Оптичес...",None,"[[0.66954195, 1.0643557, 0.78324044, -0.338267...","[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719...","{""Тип аксессуара"":[""Набор микропрепаратов""],""Б..."


In [10]:
etl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457063 entries, 0 to 457062
Data columns (total 8 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   variantid                          457063 non-null  int64 
 1   name                               457063 non-null  object
 2   categories                         457063 non-null  object
 3   color_parsed                       378652 non-null  object
 4   pic_embeddings_resnet_v1           303467 non-null  object
 5   main_pic_embeddings_resnet_v1      457063 non-null  object
 6   name_bert_64                       457063 non-null  object
 7   characteristic_attributes_mapping  457036 non-null  object
dtypes: int64(1), object(7)
memory usage: 27.9+ MB


In [11]:
etl["categories"].apply(lambda x: json.loads(x)["1"]).unique()

array(['EPG'], dtype=object)

In [12]:
etl["categories"].apply(lambda x: json.loads(x)["2"]).unique()

array(['Электроника', 'Автотовары', 'Хобби и творчество',
       'Строительство и ремонт', 'Спорт и отдых', 'Канцелярские товары',
       'Бытовая химия', 'Дом и сад', 'Детские товары', 'Бытовая техника',
       'Товары для взрослых', 'Галантерея и украшения',
       'Товары для животных'], dtype=object)

In [13]:
# решил для каждого типа фичей посмотреть на несколько примеров
from collections import defaultdict 

d = defaultdict(list)

for i in [0, 100, 1000, 10000]:
    for key in etl:
        d[key].append(etl[key][i])

In [14]:
d["categories"][0]

'{"1": "EPG", "2": "Электроника", "3": "Сетевые фильтры, разветвители и удлинители", "4": "Сетевой фильтр, удлинитель, разветвитель"}'

In [15]:
# Это я так на всякий случай убедился, что у каждого товара ровно один эмбеддинг main_pic_embeddings_resnet_v1
len(etl[etl["main_pic_embeddings_resnet_v1"].apply(lambda x: x.shape[0]) != 1])

0

In [16]:
# В тренировочном датасете у товара от 0 до 15 картинок не считая главную
amounts_of_pictures = etl["pic_embeddings_resnet_v1"].apply(lambda x: x.shape[0] if x is not None else 0).unique()
amounts_of_pictures

array([ 0,  5,  7,  2,  4,  3,  1,  6,  8, 10,  9, 12, 14, 13, 11, 15],
      dtype=int64)

In [20]:
test_dataset = pd.read_parquet("hackathon_files_for_participants_ozon/test_pairs_wo_target.parquet")
test_dataset.head(2)

,variantid1,variantid2
0,52076340,290590137
1,64525522,204128919
